# Creating a transaction with multiple inputs and outputs

In this section we'll create a P2WPKH transaction with multiple inputs and outputs. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

This section assumes knowledge of creating a P2WPKH transaction with a single input.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - TestShell setup - '[P2PKH chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb)'


## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24](https://github.com/bitcoin/bitcoin/releases/tag/v24.0).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:

path_to_bitcoin_functional_test = "/Users/dariuscognac/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/dariuscognac/Documents/Github/bitcoin-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

### Scenario

Andreas and Lisa both want to support three charities, all in foreign countries. The charities accept bitcoin donations, so they decide to send bitcoin jointly in a single transaction. Using a single transaction has a couple of advantages over each of them creating their own transactions:
- The single transaction is smaller than two separate ones, saving Andreas and Lisa some btc on transaction fees.
- The charities benefit by receiving one UTXO rather than two making it cheaper to spend from. 

Andreas has a UTXO for 0.3 btc, of which he wants to donate all of it. Lisa has a UTXO for 0.401 and she wants to match Andreas's donation of 0.3 btc. They want to donate 0.2 btc to the following addresses:
`bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`, `bcrt1q6mlqttg852e63uahyglwla55xusryqp08vx9w2`, `bcrt1qe9y40n9uwzh34mzj02w3xx9zkhgke6wxcql4lk`


### Create Andreas and Anita's P2WPKH UTXOs

We'll use bitcoind to create Andreas and Anita's P2WPKH UTXOs. Within this tutorial, to differentiate the variables for Andreas and Lisa by using the suffixes `_a` and `_l` respectively.

In [2]:
# Create two private keys and P2WPKH addresses
privkey_a = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
pubkey_a = privkey_to_pubkey(privkey_a)
p2wpkh_addr_a = pk_to_p2wpkh(pubkey_a, network = "regtest")

privkey_l = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
pubkey_l = privkey_to_pubkey(privkey_l)
p2wpkh_addr_l = pk_to_p2wpkh(pubkey_l, network = "regtest")

# Setup bitcoind and fund addresses
node = setup_testshell()
txid_a, index_a = fund_address(node, p2wpkh_addr_a, 0.3)
txid_l, index_l = fund_address(node, p2wpkh_addr_l, 0.401)

print(f"Andreas's UTXO: {txid_a}, {index_a}")
print(f"Lisa's UTXO: {txid_l}, {index_l}")

2022-12-08T10:29:45.807000Z TestFramework (INFO): Initializing test directory /var/folders/r5/yk8yg2xs1gs8xzkn5l8vr72w0000gn/T/bitcoin_func_test_ezrqykdy
Andreas's UTXO: 5b9c4d176b42349dc2a8cca26d52a3b87c9f8773dd9cacc87003eb4a96cf7044, 0
Lisa's UTXO: 050746ee1a49d3273e9819cce6bfa3b8599e46b96b0225c07ceba0ef0e92afd2, 0


### Define our outputs

Since Lisa is only donating 0.3 btc of her 0.401 btc utxo (and 0.001 btc is going towards the mining fee), Lisa wants to send the remaning 0.1 btc to her change address `bcrt1qqde3c4pmvrr9d3pav3v6hlpp9l3sm6rxnj8dcm`.

Below we'll decode the output addresses so that we know the output scriptPubkeys. For more on decoding addresses, refer back to the 'Addresses' chapter.

In [3]:
# Lisa's change output
lisa_change_value_sat = int(float("0.1") * 100_000_000)
lisa_change_spk = bech32_to_spk('bcrt', 'bcrt1qqde3c4pmvrr9d3pav3v6hlpp9l3sm6rxnj8dcm')

# Outputs to charities
output1_value_sat = int(float("0.2") * 100_000_000)
output1_spk = bech32_to_spk('bcrt', 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw')

output2_value_sat = int(float("0.2") * 100_000_000)
output2_spk = bech32_to_spk('bcrt', 'bcrt1q6mlqttg852e63uahyglwla55xusryqp08vx9w2')

output3_value_sat = int(float("0.2") * 100_000_000)
output3_spk = bech32_to_spk('bcrt', 'bcrt1qe9y40n9uwzh34mzj02w3xx9zkhgke6wxcql4lk')

### Create an unsigned transaction

As with the previous chapters, we'll fill in all the fields for our unsigned transaction but with our multiple inputs and outputs.

In [4]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have 2 input
input_count = bytes.fromhex("02")

# Convert txid and index to bytes (little endian)
txid_a = (bytes.fromhex(txid_a))[::-1]
index_a = index_a.to_bytes(4, byteorder="little", signed=False)

txid_l = (bytes.fromhex(txid_l))[::-1]
index_l = index_l.to_bytes(4, byteorder="little", signed=False)


# For the unsigned transaction we use an empty scriptSig
scriptsig_a = bytes.fromhex("")
scriptsig_l = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence_a = bytes.fromhex("ffff ffff")
sequence_l = bytes.fromhex("ffff ffff")

inputs = (
    txid_a
    + index_a
    + varint_len(scriptsig_a)
    + scriptsig_a
    + sequence_a
    + txid_l
    + index_l
    + varint_len(scriptsig_l)
    + scriptsig_l
    + sequence_l
)

# OUTPUTS
# 0x04 for four outputs
output_count = bytes.fromhex("04")

# Convert the values to 8 length byte arrays
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)
output3_value = output3_value_sat.to_bytes(8, byteorder="little", signed=False)
lisa_change_value = lisa_change_value_sat.to_bytes(8, byteorder="little", signed=False)

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
    + output3_value
    + varint_len(output3_spk)
    + output3_spk
    + lisa_change_value
    + varint_len(lisa_change_spk)
    + lisa_change_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  02000000024470cf964aeb0370c8ac9cdd73879f7cb8a3526da2cca8c29d34426b174d9c5b0000000000ffffffffd2af920eefa0eb7cc025026bb9469e59b8a3bfe6cc19983e27d3491aee4607050000000000ffffffff04002d310100000000160014fc7250a211deddc70ee5a2738de5f07817351cef002d310100000000160014d6fe05ad07a2b3a8f3b7223eeff694372032002f002d310100000000160014c94957ccbc70af1aec527a9d1318a2b5d16ce9c6809698000000000016001403731c543b60c656c43d6459abfc212fe30de86600000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [5]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "40c6bbde2a1f2f3ea4a3cc8986751fd452b91edb12684b92d853b4a8efa97bb0",
  "hash": "40c6bbde2a1f2f3ea4a3cc8986751fd452b91edb12684b92d853b4a8efa97bb0",
  "version": 2,
  "size": 216,
  "vsize": 216,
  "weight": 864,
  "locktime": 0,
  "vin": [
    {
      "txid": "5b9c4d176b42349dc2a8cca26d52a3b87c9f8773dd9cacc87003eb4a96cf7044",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    },
    {
      "txid": "050746ee1a49d3273e9819cce6bfa3b8599e46b96b0225c07ceba0ef0e92afd2",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.20000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f07817351cef",
        "address": "bcrt1ql3e9pgs3mmwuwrh9

First we'll create Andreas's signature.

In [6]:
pk_hash_a = hash160(pubkey_a)
scriptcode_a = bytes.fromhex("76a914" + pk_hash_a.hex() + "88ac")

input_amount_sat_a = int(0.3 * 100_000_000)
value_a = input_amount_sat_a.to_bytes(8, byteorder="little", signed=False)

# Now that we have multiple inputs, and we are using SIGHASH_ALL, we need to include them in
# the sighash calculation under hashPrevOuts and hashSequence
hashPrevOuts = hash256(
    txid_a
    + index_a
    + txid_l
    + index_l)
hashSequence = hash256(sequence_a + sequence_l)

hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage_a = (
    version
    + hashPrevOuts
    + hashSequence
    + txid_a
    + index_a
    + varint_len(scriptcode_a)
    + scriptcode_a
    + value_a
    + sequence_a
    + hashOutputs
    + locktime
    + sighash_type
)
sighash_a = hash256(tx_digest_preimage_a)

# Sign the sigHash with the input private key
signing_key_a = ecdsa.SigningKey.from_string(privkey_a, curve=ecdsa.SECP256k1) 
signature_a = signing_key_a.sign_digest(sighash_a, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature_a = signature_a + bytes.fromhex("01")

print("Andreas's signature: ", signature_a.hex())

Andreas's signature:  30450221009bc6239ba94cda692c0e0903af911ff2e51056f9a42f323b835c10c5488ed0390220570a587a79b9824d378a7c7a916ac973db1efe4db5e1204084e0db616ba818b701


Andreas can send Lisa his signature, confident that it can only be used in this exact transaction he's signed over. He has this guarantee because he signed over all the fields of the transaction. If any of these fields of the transaction were to change, his signature would become invalidated. In the next section we'll cover how one can use different sighash flags to sign over only specific parts of a transaction.

Now we'll create Lisa's signature.

In [7]:
pk_hash_l = hash160(pubkey_l)
scriptcode_l = bytes.fromhex("76a914" + pk_hash_l.hex() + "88ac")

input_amount_sat_l = int(0.401 * 100_000_000)
value_l = input_amount_sat_l.to_bytes(8, byteorder="little", signed=False)

tx_digest_preimage_l = (
    version
    + hashPrevOuts
    + hashSequence
    + txid_l
    + index_l
    + varint_len(scriptcode_l)
    + scriptcode_l
    + value_l
    + sequence_l
    + hashOutputs
    + locktime
    + sighash_type
)
sighash_l = hash256(tx_digest_preimage_l)

# Sign the sigHash with the input private key
signing_key_l = ecdsa.SigningKey.from_string(privkey_l, curve=ecdsa.SECP256k1) 
signature_l = signing_key_l.sign_digest(sighash_l, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature_l = signature_l + bytes.fromhex("01")

print("Lisa's signature: ", signature_l.hex())

Lisa's signature:  3045022100968fefff098f7752c4ad1af641d41ab72afb8f1be4f1ce016dc1bdbbc65d86c702202a152e8d57bc9afdb0ca9f87e8376096a260d6448097f06610060227de08133601


Now all we need to do is add the signatures to the witness field. Since we have two inputs, it's important to include the witness elements in the same order as the inputs. Each input has two witness elements, the signature and pubkey.

In [8]:
# Witness field
witness = (
    # First (Andreas's) input
    bytes.fromhex("02")
    + pushbytes(signature_a)
    + pushbytes(pubkey_a)
    # Second (Lisa's) input
    + bytes.fromhex("02")
    + pushbytes(signature_l)
    + pushbytes(pubkey_l)
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  020000000001024470cf964aeb0370c8ac9cdd73879f7cb8a3526da2cca8c29d34426b174d9c5b0000000000ffffffffd2af920eefa0eb7cc025026bb9469e59b8a3bfe6cc19983e27d3491aee4607050000000000ffffffff04002d310100000000160014fc7250a211deddc70ee5a2738de5f07817351cef002d310100000000160014d6fe05ad07a2b3a8f3b7223eeff694372032002f002d310100000000160014c94957ccbc70af1aec527a9d1318a2b5d16ce9c6809698000000000016001403731c543b60c656c43d6459abfc212fe30de866024830450221009bc6239ba94cda692c0e0903af911ff2e51056f9a42f323b835c10c5488ed0390220570a587a79b9824d378a7c7a916ac973db1efe4db5e1204084e0db616ba818b70121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa02483045022100968fefff098f7752c4ad1af641d41ab72afb8f1be4f1ce016dc1bdbbc65d86c702202a152e8d57bc9afdb0ca9f87e8376096a260d6448097f06610060227de081336012102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2700000000


### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [9]:
new_tx_txid = node.sendrawtransaction(signed_tx.hex())
# result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])print(new_tx_txid)

40c6bbde2a1f2f3ea4a3cc8986751fd452b91edb12684b92d853b4a8efa97bb0


We can decode the serialized transaction using ```decoderawtransaction```. Notice that when we parse the transaction in bitcoind, it matches the witness elements to the inputs based on the order they're entered.

Next, we'll see how we can take advantage of the different sighash flags to allow some flexibility in the transaction after one of the inputs has been signed.

In [10]:
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "40c6bbde2a1f2f3ea4a3cc8986751fd452b91edb12684b92d853b4a8efa97bb0",
  "hash": "d98f73bab0e1e5adee78c61d95ddc12f249116b73bb9005a3bbb3c3717ef56a2",
  "version": 2,
  "size": 434,
  "vsize": 271,
  "weight": 1082,
  "locktime": 0,
  "vin": [
    {
      "txid": "5b9c4d176b42349dc2a8cca26d52a3b87c9f8773dd9cacc87003eb4a96cf7044",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "30450221009bc6239ba94cda692c0e0903af911ff2e51056f9a42f323b835c10c5488ed0390220570a587a79b9824d378a7c7a916ac973db1efe4db5e1204084e0db616ba818b701",
        "034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      ],
      "sequence": 4294967295
    },
    {
      "txid": "050746ee1a49d3273e9819cce6bfa3b8599e46b96b0225c07ceba0ef0e92afd2",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3045022100968fefff098f7752c4ad1af641d41ab72afb8f1be4f1ce016dc1bdbbc65d86

## Quiz
1. Who paid for the fee in this transaction?
2. The last byte of the signatures is the sighash type flag, in this case `0x01` which corresponds to `SIGHASH_ALL`. What would prevent Lisa from simply changing this last byte and using Andreas's signature in a different transaction?
3. After Andreas sends Lisa his signature, he changes his mind and decides he wants to keep his bitcoin. Is it too late? What could he do to prevent Lisa from spending his input?
4. In the witness field, we did not explicitly state which inputs the signatures and pubkeys corresponded to. How did bitcoind know which inputs each one belonged to?

 ## Answers
1. Lisa paid 0.001 btc in fees for the transaction.
2. The sighash flag is also included in the sighash calculation. When the bitcoin node attempts to validate the signature, it'll create a sighash using the sighash type from the last byte of the signature. If the sighash flag on the signature is doesn't match the one that was used to create the signed sighash, the resulting signature would not pass.
3. Andreas could create a new transaction to spend his own input and broadcast that first.
4. The order of the witness fields must correspond to the order of the inputs in the 'inputs' field.

## Exercise

Andreas discovers he has another UTXO for 0.03 btc that he wants to include in this transaction. Use the `fund_address` function to create another UTXO and try including that in the transaction such that it has 3 inputs in total. You may want to re-use Andreas's address for convenience.